<div style="text-align: center;">
    <h1> Tensorflow 2.0 - Model prototyping for DL4J</h1>
    <div style="display: inline-block">
        <img src="https://avatars2.githubusercontent.com/u/15658638?s=400&v=4" height="100" width="100"> 
    </div>
    <div style="display: inline-block">
        <img src="https://devblogs.nvidia.com/wp-content/uploads/2017/08/Keras_Logo_358x230.png" height="150" width="150">
    </div>
    <div style="display: inline-block">
        <img src="https://commons.bmstu.wiki/images/4/46/DL4J.png" height="200" width="200">
    </div>
</div>

In [ ]:
import json
import numpy as np
import urllib.request
import glob
import os
from os.path import join as ospj
import matplotlib.pyplot as plt
import matplotlib
from random import randint
%matplotlib inline 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow import image
from tensorflow import data

# Download npy

In [ ]:
!mkdir data
f = open("classes.txt","r")
# And for reading use
classes = f.readlines()
f.close()

def download():
    base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
    for c in classes:
        clean_class = c.strip()
        path = base + clean_class + '.npy'
        print(path)
        urllib.request.urlretrieve(path, 'data/' + clean_class + '.npy')

In [ ]:
#download()


# Data Processing

In [ ]:
def load_data(root, vfold_ratio=0.2, max_items_per_class=4000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables 
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    #load each data file 
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None
    
    #randomize the dataset 
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    #separate into training and testing 
    vfold_size = int(x.shape[0] / 100 * (vfold_ratio * 100))

    x_test = x[0 : vfold_size, :]
    y_test = y[0: vfold_size]

    x_train = x[vfold_size : x.shape[0], :]
    y_train = y[vfold_size : y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names


In [ ]:
x_train, y_train, x_test, y_test, class_names = load_data('data')
num_classes = len(class_names)
image_size = 28

In [ ]:
idx = randint(0, len(x_train))
plt.imshow(x_train[idx].reshape(28,28), cmap='gray_r') 
print(class_names[int(y_train[idx].argmax())])

In [ ]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Building network

In [ ]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax')) 
# Train model
adam = tf.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

# Fit 

In [ ]:
model.fit(x=x_train, y=y_train, validation_split=0.1, batch_size=256, verbose=2, epochs=5)

# Testing

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

In [ ]:
idx = randint(0, len(x_test))
img = x_test[idx]
plt.imshow(img.squeeze(), cmap='gray_r') 
pred = model.predict(np.expand_dims(img, axis=0))[0]
ind = (-pred).argsort()[:5]
probas = [pred[x] for x in ind]
latex = [class_names[x] for x in ind]
print(list(zip(latex, probas)))

# Persist

In [ ]:
model.save('../data/models/drawmeacatNet.h5')